## ספריות

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
# import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
# import adjustText as aT

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### העלת משתנים להרצת הקוד

In [3]:
path = os.getcwd()

parent = os.path.dirname(path)

software_data_folder_location = os.path.dirname(parent)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_data_folder_location)

### פונקציות

In [4]:
from py_scripts.layer_of_buildings import layer_of_buildings
from py_scripts.remove_irrelevant_buildings import remove_irrelevant_buildings

### פונקציות גלובליות

In [5]:
from functions import double_taz_num
from functions import make_point
from functions import up_load_df
from functions import up_load_gdb
from functions import up_load_shp

In [6]:
# def double_taz_num(df):
#     dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
#     return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [7]:
# def make_point(df):
#     df_point=df.copy()
#     df_point['centroid'] = df_point.representative_point()
#     df_point=df_point.set_geometry('centroid')
#     df_point=df_point.drop(columns=['geometry'],axis=1)
#     return df_point

In [8]:
# def up_load_gdb(path,layer_name):
#     path='{}'.format(path)
#     layer_list=fiona.listlayers(path)
#     gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
#     return gpd_layer

In [9]:
# def up_load_shp(path):
#     path='{}'.format(path)
#     gpd_layer=gpd.read_file(path)
#     return gpd_layer

In [10]:
# def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
#     code_to_find=list(df_to_geoode[unique_field].unique())

#     gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

#     gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

#     gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

#     return gpd_for_geocode_in_one_taz

In [11]:
# def up_load_df(folder_path,file_name):
#     path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
#     df=pd.read_excel(path_df)
#     df=df.dropna(how='all')

#     return df


## 

### העלת מידע

In [12]:
# #העלת משתנים להרצת הקוד
# df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

# software_data_folder_location=df_inputs_outputs['location'][0]

# forecast_version_folder_location=df_inputs_outputs['location'][1]


In [13]:
stat=up_load_shp(r'{}\background_files\stat_11_cbs_2020_with_jtmt_fix.shp'.format(software_data_folder_location))

In [14]:
stat_in_taz=stat.loc[stat['type_stat']=='in_taz']

In [15]:
stat=stat.loc[stat['type_stat']!='in_taz']

In [16]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

In [17]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_data_folder_location)
bld=up_load_gdb(path,'BLDG_Clip')

In [18]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_data_folder_location)
bld_poi=up_load_gdb(path,'POI_BLDG_Clip')

In [19]:
taz_not_relevant=up_load_df(r'{}\background_files'.format(software_data_folder_location),'taz_not_relevant_for_pop_2020')

In [20]:
taz_not_relevant=list(taz_not_relevant.Taz_num)

### שכבת בניינים

In [21]:
bld_point=layer_of_buildings(bld, bld_poi)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\from_sa_cbs_to_ta_jtmt\py_scripts\layer_of_buildings.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 31.3225      20.58816099  23.03316663 ...  79.56155771  86.00171698
 411.10230836]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']


### הורדת מבנים לא רלוונטים

In [26]:
bld_point=remove_irrelevant_buildings(bld_point, taz, taz_not_relevant, stat)

### יצירת מאה אחוז לכל א"ס

In [27]:
sum_bld_m3=pd.pivot_table(bld_point,index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()

C:\Users\dpere\AppData\Local\Temp\ipykernel_29344\1821465286.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sum_bld_m3=pd.pivot_table(bld_point,index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()


TypeError: agg function failed [how->sum,dtype->geometry]

In [ ]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat'}).reset_index()

In [ ]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [ ]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat']

In [ ]:
sum_bld_m3=sum_bld_m3.loc[(sum_bld_m3['precent_of_stat_data']>0.01)|(sum_bld_m3['Taz_num']==2001)]

In [ ]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat_new'}).reset_index()

In [ ]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [ ]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat_new']

### א"ס ללא פוליגון



In [ ]:
stat_in_taz=make_point(stat_in_taz).sjoin(taz[['Taz_num','geometry']])[['STAT','Taz_num']]

In [ ]:
stat_in_taz['precent_of_stat_data']=1

### איחוד לטבלה אחת



In [ ]:
col=['Taz_num', 'STAT',  'precent_of_stat_data']

In [ ]:
precent_of_stat_data=pd.concat([sum_bld_m3[col], stat_in_taz])

In [ ]:
precent_of_stat_data.to_excel(r'{}\Intermediates\precent_of_stat_data.xlsx'.format(software_data_folder_location),index=False)

###  יצירת מידע ברמת אזור תנועה

In [ ]:
stat=up_load_df(r'{}\Intermediates'.format(software_data_folder_location),'stat_cbs_jtmt_2020')

In [ ]:
col=['pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [ ]:
stat[col]=stat[col].multiply(stat['pop'], axis="index")

In [ ]:
stat['pop_hardi']=stat['pop']*(stat['pre_hardi']/100)

In [ ]:
taz_stat_conver=precent_of_stat_data

In [ ]:
col=['STAT',
 'aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

בשכבת האזורי תנועה יהיו רק אזורי תנועה שמפצלים את המידע של למס

In [ ]:
taz=pd.merge(taz_stat_conver,stat[col],on='STAT',how='left')

In [ ]:
col=['aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

In [ ]:
taz[col]=taz[col].multiply(taz['precent_of_stat_data'], axis="index")

In [ ]:
taz=pd.pivot_table(taz,index='Taz_num',aggfunc=sum)

In [ ]:
taz['pre_hardi']=taz['pop_hardi']/taz['pop']

In [ ]:
taz['hh_size']=taz['pop']/taz['aprt_20']

In [ ]:
save_taz_path=r'{}\Intermediates'.format(software_data_folder_location)

In [ ]:
taz.to_excel(r'{}\taz_with_pop_info.xlsx'.format(save_taz_path))